## Creating the environment

In [ ]:
class TTT:
    def __init__(self):
        state = [0] * 9

    def make_move(self, player : int, ind : int):
        if (player != 1 and player != 2) or (ind < 0 or ind > 8):
            print(f"Invalid player {player} or index {ind} in make move")
            return
        
        if (self.state[ind] == 0):
            self.state[ind] = player
        else: 
            print(f"{ind} index has value of player {self.state[ind]}")
    
    def win_found(self, ind : int, player : int): #returns boolean
        col = ind % 3
        row = ind / 3

        for i in range(3):
            if (self.state[row + i] != player or self.state[col + 3*i] != player):
                return False
            
        # still left for diagonals
        # Consider both left and right diagonals
        
        return True